### Финальный проект

Мы уже прошли всю необходимую теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - money precision@5. Порог для уcпешной сдачи проекта money precision@5 > 20%

Бизнес ограничения в топ-5 товарах:
- Для каждого юзера 5 рекомендаций (иногда модели могут возвращать < 5)
- **2 новых товара** (юзер никогда не покупал)
- **1 дорогой товар, > 7 долларов**
- **Все товары из разных категорий** (категория - sub_commodity_desc)  
- **Стоимость каждого рекомендованного товара > 1 доллара**  

- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и .csv файл с рекомендациями. В .csv файле 2 столбца: user_id - (item_id1, item_id2, ..., item_id5)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items


# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

//anaconda3/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
# Настройки для удобства
os.environ['MKL_NUM_THREADS'] = '1'
pd.options.display.float_format = "{:.5f}".format
pd.set_option('display.max_colwidth', 100)

In [3]:
data = pd.read_csv(module_path+'/retail_train.csv')
data.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39000,364,-0.60000,1631,1,0.00000,0.00000
1,2375,26984851472,1,1033142,1,0.82000,364,0.00000,1631,1,0.00000,0.00000
2,2375,26984851472,1,1036325,1,0.99000,364,-0.30000,1631,1,0.00000,0.00000
3,2375,26984851472,1,1082185,1,1.21000,364,0.00000,1631,1,0.00000,0.00000
4,2375,26984851472,1,8160430,1,1.50000,364,-0.39000,1631,1,0.00000,0.00000


In [4]:
data.shape

(2396804, 12)

In [5]:
item_features = pd.read_csv(module_path+'/product.csv')
item_features = item_features.rename({'PRODUCT_ID': 'item_id'}, axis='columns')
item_features = item_features.rename(str.lower, axis='columns')

item_features.head(5)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [6]:
user_features = pd.read_csv(module_path+'/hh_demographic.csv')
user_features = user_features.rename({'household_key': 'user_id'}, axis='columns')
user_features = user_features.rename(str.lower, axis='columns')
user_features.head(5)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [7]:
# добавим price в item_features

quantity = data.groupby('item_id')['quantity'].sum()
sales_value = data.groupby('item_id')['sales_value'].sum()
price = sales_value/quantity
item_features = item_features.join(price.rename('price'), on='item_id')

item_features.head()

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,price
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB,3.49000
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,,0.99000
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,,1.59000
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ,1.54000
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ,0.99000


In [8]:
# добавим price в data
data = data.join(item_features[['item_id', 'price']].set_index('item_id'), on='item_id')

# добавим sub_commodity_desc в транзакции
data = data.join(item_features[['item_id', 'sub_commodity_desc']].set_index('item_id'), on='item_id')

# добавим dataweek_weighted_sales_value дата
# инверсия недель: 95 неделя становится 1
data['week_ago'] = data['week_no'].max() - data['week_no'] + data['week_no'].min()
alpha = 0.9527 # 1 прошлый доллар превращается в 1 цент
data['week_weighted_sales_value'] = data['sales_value'] * (alpha**data['week_ago'])

# оставим data с price > 1 доллара
data = data[data['price'] > 1]

data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price,sub_commodity_desc,week_ago,week_weighted_sales_value
0,2375,26984851472,1,1004906,1,1.39000,364,-0.60000,1631,1,0.00000,0.00000,2.38518,POTATOES RUSSET (BULK&BAG),95,0.01393
2,2375,26984851472,1,1036325,1,0.99000,364,-0.30000,1631,1,0.00000,0.00000,1.31671,CELERY,95,0.00992
4,2375,26984851472,1,8160430,1,1.50000,364,-0.39000,1631,1,0.00000,0.00000,1.68268,ORGANIC CARROTS,95,0.01503
6,2375,26984851516,1,1043142,1,1.57000,364,-0.68000,1642,1,0.00000,0.00000,2.39478,SPONGES: BATH HOUSEHOLD,95,0.01573
7,2375,26984851516,1,1085983,1,2.99000,364,-0.40000,1642,1,0.00000,0.00000,2.91560,TRAY PACK/CHOC CHIP COOKIES,95,0.02996


In [9]:
# popularity сгруппированные по юзерам, отсортированные по week_weighted_sales_value
popularity_by_users = data.groupby(['user_id', 'item_id'])[['week_weighted_sales_value']].sum()\
    .sort_values(by=['user_id', 'week_weighted_sales_value'], ascending=False).reset_index()\
    .join(item_features[['item_id', 'sub_commodity_desc', 'price']].set_index('item_id'), on='item_id')
# popularity[popularity['user_id']==2499]
popularity_by_users.head()

,user_id,item_id,week_weighted_sales_value,sub_commodity_desc,price
0,2500,1065538,103.50775,BEERALEMALT LIQUORS,15.77175
1,2500,6979673,38.54940,INFANT FORMULA SOY BASE,11.43444
2,2500,8118676,22.68186,BABY DIAPERS,27.38583
3,2500,9803352,19.04447,INFANT FORMULA TODDLER,19.95667
4,2500,948670,12.85137,TICKETS,8.97887


In [10]:
# Актуальные продажи с quantity=>0
result = data.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

# Актуальные продажи с quantity>0
actual_with_positive_quantity = data[data['quantity']>0].groupby('user_id')['item_id'].unique()
result = result.join(actual_with_positive_quantity.rename('actual_with_positive_quantity'), on='user_id')

# Добавим в актуальные продажи sorted_week_weighted_sales_value
sorted_week_weighted_sales_value = popularity_by_users.groupby('user_id')['item_id'].unique()
result = result.join(sorted_week_weighted_sales_value.rename('sorted_week_weighted_sales_value'), on='user_id')

result.head(2)

,user_id,actual,actual_with_positive_quantity,sorted_week_weighted_sales_value
0,1,"[825123, 831447, 845307, 852014, 854982, 856942, 912676, 940947, 945805, 958046, 977545, 991024,...","[825123, 831447, 845307, 852014, 854982, 856942, 912676, 940947, 945805, 958046, 977545, 991024,...","[856942, 8293439, 940947, 9527558, 10149640, 1005186, 5577022, 1049998, 8090541, 9297615, 995242..."
1,2,"[854852, 930118, 1077555, 5567388, 5567582, 5568489, 5569230, 9365106, 846833, 978318, 899824, 9...","[854852, 930118, 1077555, 5567388, 5567582, 5568489, 5569230, 9365106, 846833, 978318, 899824, 9...","[916122, 16035203, 8090521, 15572067, 1134678, 970481, 1106523, 1073678, 899624, 5569230, 110809..."


In [11]:
# уникальные товары из data
items_from_data = pd.DataFrame(data['item_id'].unique(), columns=['item_id'])
items_from_data.shape

(76798, 1)

In [12]:
### в item_features

# товар, который больше покупали (кол-во продаж)
total_sales_quantity = data.groupby('item_id')['quantity'].sum()

# товар у которого больше всего уникальных покупателей (кол-во юзеров у товара)
nunique_buyers = data.groupby(['item_id'])['user_id'].nunique()

# товар, который чаще всего покупали
total_sales_freq = data.groupby('item_id')['quantity'].count() 

# repurchase_number сколько раз купил тот же юзер
repurchase_number = data.groupby(['user_id', 'item_id'])[['quantity']].count()
repurchase_number = repurchase_number[repurchase_number.quantity>1].groupby('item_id')['quantity'].sum()

# repeat_customer_number Сколько пользователей купили товар больше одного раза
repeat_customer_number = data.groupby(['item_id', 'user_id'])[['quantity']].count()
repeat_customer_number = repeat_customer_number[repeat_customer_number.quantity>1]
repeat_customer_number = repeat_customer_number.reset_index().groupby(['item_id'])['user_id'].count()

# week_weighted_sales_quantity давность покупки  (сумма продаж за неделю на альфа в степени week_n)
week_weighted_sales_quantity = data.groupby(['item_id', 'week_no'])['quantity'].sum()
week_weighted_sales_quantity = week_weighted_sales_quantity.reset_index()
# инверсия недель: 95 неделя становится 1
week_weighted_sales_quantity['week_ago'] = week_weighted_sales_quantity['week_no'].max() - week_weighted_sales_quantity['week_no'] + week_weighted_sales_quantity['week_no'].min()
alpha = 0.90
week_weighted_sales_quantity['week_weighted_sales_quantity'] = week_weighted_sales_quantity['quantity'] * (alpha**week_weighted_sales_quantity['week_ago'])
week_weighted_sales_quantity = week_weighted_sales_quantity.groupby(['item_id'])['week_weighted_sales_quantity'].sum()

# week_weighted_sales_value
week_weighted_sales_value = data.groupby(['item_id', 'week_no'])['sales_value'].sum()
week_weighted_sales_value = week_weighted_sales_value.reset_index()
# инверсия недель: 95 неделя становится 1
week_weighted_sales_value['week_ago'] = week_weighted_sales_value['week_no'].max() - week_weighted_sales_value['week_no'] + week_weighted_sales_value['week_no'].min()
alpha = 0.90
week_weighted_sales_value['week_weighted_sales_value'] = week_weighted_sales_value['sales_value'] * (alpha**week_weighted_sales_value['week_ago'])
week_weighted_sales_value = week_weighted_sales_value.groupby(['item_id'])['week_weighted_sales_value'].sum()

# НЕ НАДО в скольких неделях товар продавался. неделя дальше вес ниже
# популярность внутри отдела, доля продаж товара внутри отдела
# юзеры, которые чаще покупают имеют больший вес (количество покупок юзером, количество уникальных покупок юзером)

In [13]:
# добавляем фичи из data в items_from_data
items_from_data = items_from_data.join(total_sales_quantity.rename('total_sales_quantity'), on='item_id')
items_from_data = items_from_data.join(nunique_buyers.rename('nunique_buyers'), on='item_id')
items_from_data = items_from_data.join(total_sales_freq.rename('total_sales_freq'), on='item_id')
items_from_data = items_from_data.join(repurchase_number.rename('repurchase_number'), on='item_id')
items_from_data = items_from_data.join(repeat_customer_number.rename('repeat_customer_number'), on='item_id')
items_from_data = items_from_data.join(week_weighted_sales_quantity.rename('week_weighted_sales_quantity'), on='item_id')
items_from_data = items_from_data.join(week_weighted_sales_value.rename('week_weighted_sales_value'), on='item_id')

items_from_data = items_from_data.set_index('item_id')
items_from_data.head()

,total_sales_quantity,nunique_buyers,total_sales_freq,repurchase_number,repeat_customer_number,week_weighted_sales_quantity,week_weighted_sales_value
item_id,,,,,,,
1004906,3675,1141,3339,2880.00000,682.00000,401.33307,1016.28728
1036325,216,159,210,82.00000,31.00000,0.03601,0.04672
8160430,209,134,193,86.00000,27.00000,27.93776,52.10429
1043142,92,78,89,22.00000,11.00000,10.24893,26.80256
1085983,343,133,297,199.00000,35.00000,21.16892,69.95808


In [14]:
# считаем общий score (вес week_weighted_sales_value *2 – захотелось)

# MinMaxScaler() фич
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(items_from_data)
items_from_data.loc[:, :] = scaled_values

items_from_data['score'] = items_from_data['total_sales_quantity'] + items_from_data['nunique_buyers'] + items_from_data['total_sales_freq']\
+ (0 if items_from_data['repurchase_number'].isnull else items_from_data['repurchase_number'])\
+ (0 if items_from_data['repeat_customer_number'].isnull else items_from_data['repeat_customer_number']) + items_from_data['week_weighted_sales_quantity']\
+ 2*items_from_data['week_weighted_sales_value']

items_from_data.describe()

,total_sales_quantity,nunique_buyers,total_sales_freq,repurchase_number,repeat_customer_number,week_weighted_sales_quantity,week_weighted_sales_value,score
count,76798.00000,76798.00000,76798.00000,30367.00000,30367.00000,76798.00000,76798.00000,76798.00000
mean,0.00152,0.00828,0.00176,0.00258,0.00695,0.00175,0.00208,0.01747
std,0.00914,0.02679,0.01010,0.01431,0.02549,0.01023,0.01047,0.07183
min,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
25%,0.00005,0.00000,0.00000,0.00008,0.00000,0.00000,0.00001,0.00020
50%,0.00015,0.00063,0.00015,0.00038,0.00086,0.00011,0.00016,0.00218
75%,0.00076,0.00569,0.00089,0.00175,0.00516,0.00084,0.00130,0.01122
max,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,5.60470


In [15]:
# создаем top_5 для рекомендаций новым юзерам. Страшный, конечно, код. Но это же MVP
top_items = items_from_data.sort_values(by='score', ascending=False)[['score']]\
.join(item_features[['item_id', 'sub_commodity_desc', 'price']].set_index('item_id'), on='item_id').reset_index()\
.groupby('sub_commodity_desc').first().sort_values(by='score', ascending=False)

# Сначала выбрали топ item >7$
top_5 = top_items[top_items['price']>7].head(1)['item_id'].to_list()

# Отфильтровали и добавили еще 4 item
top_5 = top_5+(top_items.loc[~top_items['item_id'].isin(top_5), 'item_id'].head(4).to_list())

top_5

[6533765, 1029743, 981760, 916122, 5569230]

In [16]:
# Популярные товары popularity_by_score, берем >mean()*2.5
popularity_by_score = items_from_data[items_from_data['score']>items_from_data['score'].mean()*2.5].sort_values(by='score', ascending=False).index.to_list()
print(f'Было – ', items_from_data.shape[0])
print(f'Стало – ', len(popularity_by_score))

Было –  76798
Стало –  7277


In [17]:
# добавим score в data
data = data.join(items_from_data['score'], on='item_id')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price,sub_commodity_desc,week_ago,week_weighted_sales_value,score
0,2375,26984851472,1,1004906,1,1.39000,364,-0.60000,1631,1,0.00000,0.00000,2.38518,POTATOES RUSSET (BULK&BAG),95,0.01393,1.83361
2,2375,26984851472,1,1036325,1,0.99000,364,-0.30000,1631,1,0.00000,0.00000,1.31671,CELERY,95,0.00992,0.12635
4,2375,26984851472,1,8160430,1,1.50000,364,-0.39000,1631,1,0.00000,0.00000,1.68268,ORGANIC CARROTS,95,0.01503,0.14768
6,2375,26984851516,1,1043142,1,1.57000,364,-0.68000,1642,1,0.00000,0.00000,2.39478,SPONGES: BATH HOUSEHOLD,95,0.01573,0.07757
7,2375,26984851516,1,1085983,1,2.99000,364,-0.40000,1642,1,0.00000,0.00000,2.91560,TRAY PACK/CHOC CHIP COOKIES,95,0.02996,0.16603


In [18]:
# Разбиваем сет
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price,sub_commodity_desc,week_ago,week_weighted_sales_value,score
0,2375,26984851472,1,1004906,1,1.39000,364,-0.60000,1631,1,0.00000,0.00000,2.38518,POTATOES RUSSET (BULK&BAG),95,0.01393,1.83361
2,2375,26984851472,1,1036325,1,0.99000,364,-0.30000,1631,1,0.00000,0.00000,1.31671,CELERY,95,0.00992,0.12635
4,2375,26984851472,1,8160430,1,1.50000,364,-0.39000,1631,1,0.00000,0.00000,1.68268,ORGANIC CARROTS,95,0.01503,0.14768
6,2375,26984851516,1,1043142,1,1.57000,364,-0.68000,1642,1,0.00000,0.00000,2.39478,SPONGES: BATH HOUSEHOLD,95,0.01573,0.07757
7,2375,26984851516,1,1085983,1,2.99000,364,-0.40000,1642,1,0.00000,0.00000,2.91560,TRAY PACK/CHOC CHIP COOKIES,95,0.02996,0.16603


In [19]:
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price,sub_commodity_desc,week_ago,week_weighted_sales_value,score
0,2375,26984851472,1,1004906,1,1.39000,364,-0.60000,1631,1,0.00000,0.00000,2.38518,POTATOES RUSSET (BULK&BAG),95,0.01393,1.83361
2,2375,26984851472,1,1036325,1,0.99000,364,-0.30000,1631,1,0.00000,0.00000,1.31671,CELERY,95,0.00992,0.12635
4,2375,26984851472,1,8160430,1,1.50000,364,-0.39000,1631,1,0.00000,0.00000,1.68268,ORGANIC CARROTS,95,0.01503,0.14768
6,2375,26984851516,1,1043142,1,1.57000,364,-0.68000,1642,1,0.00000,0.00000,2.39478,SPONGES: BATH HOUSEHOLD,95,0.01573,0.07757
7,2375,26984851516,1,1085983,1,2.99000,364,-0.40000,1642,1,0.00000,0.00000,2.91560,TRAY PACK/CHOC CHIP COOKIES,95,0.02996,0.16603


In [20]:
# Заведем фиктивный item_id (если юзер покупал товары не из popularity_by_score, то он "купил" такой товар)
data.loc[~data['item_id'].isin(popularity_by_score), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data, 
                                  index='user_id', columns='item_id', 
                                  values='score', # Можно пробоват другие варианты
                                  aggfunc='sum',
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head()

item_id,818981,819063,819112,819255,819304,819308,819330,819487,819518,819594,...,16809589,16809649,16809666,17104444,17105058,17105530,17106656,17168855,17208470,17241023
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,0.00000,0.00000,0.00000,0.26886,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
5,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [21]:
# Создаем словари
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids)) #ключ - item_id из дата
userid_to_id = dict(zip(userids, matrix_userids))

In [22]:
# Применим взвешивание
# Попробовал, результат хуже оказался
# user_item_matrix = bm25_weight(user_item_matrix.T).T 

In [29]:
%%time
# Строим модель – filter_already_liked_items=True
model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=True, #???? ВЫДАСТ ТОВАРЫ, которые не покупал?
                        filter_items=None, 
                        recalculate_user=True)


CPU times: user 12.7 s, sys: 92.2 ms, total: 12.8 s
Wall time: 13.3 s


In [30]:
def get_recommendations(user, model, N=5):
    try: 
        res = [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[user], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=N, 
                                        filter_already_liked_items=True, 
                                        filter_items=[itemid_to_id[999999]], 
                                        recalculate_user=True)]
    except:
        res = top_5
    return res

In [31]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [32]:
%%time
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

CPU times: user 1min, sys: 195 ms, total: 1min
Wall time: 1min 1s


In [33]:
%%time
# можно не запускать, так как оказывается просто model.recommend тоже ранжирует
# recommend_all делает перемножение, но еще и сортирует и выбирает топ-N

recommendations = model.recommend_all(N=5, 
                                      user_items=csr_matrix(user_item_matrix).tocsr(),
                                      filter_already_liked_items=True, 
                                      filter_items=None, 
                                      recalculate_user=True,
                                      show_progress=True,
                                      batch_size=500)


def get_ranged_recommendations(user_id):
    items = []
    for item in recommendations[userid_to_id[user_id]]:
        items.append(id_to_itemid[item])
    return items

# get_ranged_recommendations(2492)


CPU times: user 52.7 s, sys: 82.2 ms, total: 52.8 s
Wall time: 52.8 s


In [34]:
result.tail()

,user_id,actual,actual_with_positive_quantity,sorted_week_weighted_sales_value,als
2489,2491,"[975589, 1005609, 1007911, 1035143, 1070105, 1107522, 9527290, 879755, 880469, 1075275, 1133556,...","[975589, 1005609, 1007911, 1035143, 1070105, 1107522, 9527290, 879755, 880469, 1075275, 1133556,...","[999198, 890695, 991932, 838050, 1012136, 1062117, 928036, 15926863, 9527290, 995242, 880579, 11...","[9526410, 930917, 981760, 965766, 1110843]"
2490,2492,"[826860, 847207, 996833, 1037894, 1085604, 1133018, 1042438, 1063509, 1106523, 1111818, 9575546,...","[826860, 847207, 996833, 1037894, 1085604, 1133018, 1042438, 1063509, 1106523, 1111818, 9575546,...","[6533765, 1106523, 914577, 883003, 5569230, 933835, 12582494, 912004, 866755, 859426, 259120, 10...","[1092026, 1058997, 1018740, 971922, 925626]"
2491,2493,"[941360, 964219, 1095347, 1101411, 844165, 893501, 1034443, 1092026, 1131438, 913038, 828867, 84...","[941360, 964219, 1101411, 844165, 893501, 1034443, 1092026, 1131438, 913038, 828867, 845193, 856...","[926905, 1051323, 6533765, 12810455, 999779, 1038663, 859075, 13003100, 1002787, 902094, 1004906...","[979707, 893018, 1012587, 1106301, 1029624]"
2492,2494,"[825550, 860348, 870614, 890739, 896322, 914188, 914585, 923149, 942525, 968363, 981521, 1007804...","[825550, 860348, 870614, 890739, 896322, 914188, 914585, 923149, 942525, 968363, 981521, 1007804...","[889700, 12781924, 969568, 10356658, 5564636, 1133378, 1082955, 828393, 13411933, 971922, 906631...","[5569230, 951412, 918335, 9526410, 909714]"
2493,2495,"[889731, 1554796, 1567239, 1610012, 1445740, 6535008, 1637842, 8195272, 12346966, 1010308, 10325...","[889731, 1554796, 1567239, 1610012, 1445740, 6535008, 1637842, 8195272, 12346966, 1010308, 10325...","[867931, 906838, 1096603, 13007264, 1067419, 887474, 5568445, 1006184, 8019968, 918676, 846811, ...","[1058997, 847982, 995965, 923746, 5568378]"
2494,2496,"[852159, 871756, 886703, 899624, 916122, 948272, 953339, 998239, 999270, 1000753, 1020581, 10565...","[852159, 871756, 886703, 899624, 916122, 948272, 953339, 998239, 999270, 1000753, 1020581, 10565...","[916122, 1000753, 16122432, 5569230, 12810464, 12456867, 12810393, 844179, 999858, 820122, 11065...","[5592931, 12301100, 919534, 870547, 1027168]"
2495,2497,"[838220, 1037840, 1052294, 5569230, 8090537, 1022428, 5569471, 5569845, 8177622, 849202, 906625,...","[838220, 1037840, 1052294, 5569230, 8090537, 1022428, 5569471, 5569845, 8177622, 849202, 906625,...","[1010950, 995238, 1066685, 6633056, 1102207, 12812261, 1081262, 10356646, 1103513, 12731425, 102...","[898068, 1023720, 921744, 821083, 948670]"
2496,2498,"[901776, 1005186, 1031697, 1070820, 9677846, 1044500, 1049832, 5576075, 9526411, 9527290, 963234...","[901776, 1005186, 1031697, 1070820, 9677846, 1044500, 1049832, 5576075, 9526411, 9527290, 963234...","[1106523, 1070820, 1022066, 901776, 10121837, 966718, 9297516, 1130858, 1100379, 15781095, 93335...","[1058997, 1042438, 981760, 1110843, 1014948]"
2497,2499,"[838186, 853197, 883665, 932949, 933835, 1067695, 1132771, 1044655, 821344, 822346, 831763, 8358...","[838186, 853197, 883665, 932949, 933835, 1067695, 1132771, 1044655, 821344, 822346, 831763, 8358...","[1070820, 876984, 5566398, 904129, 882308, 886372, 16058930, 844179, 5570048, 1119993, 1060872, ...","[913210, 1014948, 6533765, 944139, 1038217]"
2498,2500,"[856455, 903476, 834102, 837751, 880797, 893018, 896161, 961024, 964521, 972849, 972931, 984054,...","[856455, 903476, 834102, 837751, 880797, 893018, 896161, 961024, 964521, 972849, 972931, 984054,...","[1065538, 6979673, 8118676, 9803352, 948670, 5575519, 5566235, 899019, 9837296, 14077264, 854405...","[1006184, 1136257, 12810393, 12301109, 893501]"


In [36]:
# final_top_5 c учетом требований
def final_top_5(user_id):
    try:
        # final_top и final_top_5
        als = result[result.user_id==user_id]['als'].to_list()[0]
        sorted_week_weighted_sales_value = result[result.user_id==user_id]['sorted_week_weighted_sales_value'].to_list()[0].tolist()
        # sorted_week_weighted_sales_value

        # создаем df
        final_top = pd.DataFrame(als, columns = ['item_id'])

        # вначале списка 2 item, которые юзер никогда не покупал
        final_top = final_top.join(item_features[['item_id', 'sub_commodity_desc', 'price']].set_index('item_id'), on='item_id')\
        .groupby('sub_commodity_desc', sort=False).first().head(2).reset_index()['item_id'].to_list()+sorted_week_weighted_sales_value[:50]

        # берем первый item>7 и добавляем сверху списка
        final_7_bucks_item = pd.DataFrame(final_top, columns = ['item_id']).join(item_features[['item_id', 'sub_commodity_desc', 'price']].set_index('item_id'), on='item_id')\
        .groupby('sub_commodity_desc', sort=False).first()
        final_7_bucks_item = final_7_bucks_item[final_7_bucks_item['price']>7].head(1)['item_id'].tolist()

        # собираем final_top_5
        final_top_5 = final_7_bucks_item+final_top
        final_top_5 = pd.DataFrame(final_top_5, columns = ['item_id'])
        final_top_5 = final_top_5.join(item_features[['item_id', 'sub_commodity_desc', 'price']].set_index('item_id'), on='item_id')\
        .groupby('sub_commodity_desc', sort=False).first().head(5)['item_id'].tolist()
    except:
        final_top_5 = top_5

    return final_top_5

In [37]:
%%time
result['final_top_5'] = result['user_id'].apply(lambda x: final_top_5(x))

CPU times: user 2min 30s, sys: 3.72 s, total: 2min 34s
Wall time: 2min 34s


In [38]:
result.head()

,user_id,actual,actual_with_positive_quantity,sorted_week_weighted_sales_value,als,final_top_5
0,1,"[825123, 831447, 845307, 852014, 854982, 856942, 912676, 940947, 945805, 958046, 977545, 991024,...","[825123, 831447, 845307, 852014, 854982, 856942, 912676, 940947, 945805, 958046, 977545, 991024,...","[856942, 8293439, 940947, 9527558, 10149640, 1005186, 5577022, 1049998, 8090541, 9297615, 995242...","[1100972, 1127831, 885290, 971922, 902172]","[1121590, 1100972, 1127831, 856942, 8293439]"
1,2,"[854852, 930118, 1077555, 5567388, 5567582, 5568489, 5569230, 9365106, 846833, 978318, 899824, 9...","[854852, 930118, 1077555, 5567388, 5567582, 5568489, 5569230, 9365106, 846833, 978318, 899824, 9...","[916122, 16035203, 8090521, 15572067, 1134678, 970481, 1106523, 1073678, 899624, 5569230, 110809...","[1039156, 1056509, 1065593, 897954, 981760]","[16035203, 1039156, 1056509, 916122, 8090521]"
2,3,"[866211, 878996, 882830, 921345, 931940, 937791, 951590, 964968, 1089954, 1092026, 1118235, 1121...","[866211, 878996, 882830, 921345, 931940, 937791, 951590, 964968, 1089954, 1092026, 1118235, 1121...","[958154, 1053690, 854405, 910032, 921345, 1092026, 998206, 913210, 1092937, 1074931, 15741861, 1...","[896369, 1029624, 856772, 1051211, 908531]","[958154, 896369, 1029624, 1053690, 854405]"
3,4,"[836163, 877523, 878909, 883932, 891423, 897125, 904973, 907631, 910109, 933246, 939907, 940766,...","[836163, 877523, 878909, 883932, 891423, 897125, 904973, 907631, 910109, 933246, 939907, 940766,...","[1052294, 936470, 7431408, 1021324, 13115481, 891423, 883932, 1035676, 1038692, 831063, 13115703...","[981760, 1048462, 865456, 965766, 5568729]","[1052294, 981760, 1048462, 936470, 7431408]"
4,5,"[938983, 5980822, 1012352, 825538, 1002499, 6904776, 870826, 889509, 911270, 927491, 937626, 941...","[938983, 5980822, 1012352, 825538, 1002499, 6904776, 870826, 889509, 911270, 927491, 937626, 941...","[8358766, 1065017, 1142088, 1079440, 874972, 1003031, 998793, 1101870, 950554, 1109200, 6463482,...","[981760, 1058997, 1106301, 854405, 5569327]","[8358766, 981760, 1058997, 1065017, 1142088]"


In [39]:
result.apply(lambda row: precision_at_k(row['final_top_5'], row['actual']), axis=1).mean()

0.5998732826463741

In [40]:
result[['user_id', 'final_top_5']].to_csv('predictions_final_top_5.csv', index=False)